In [ ]:
# Libraries to be used by the notebook
import json
import networkx as nx
import matplotlib.pyplot as plt
import csv
import re
import pandas as pd
import openai
import os
from dotenv import load_dotenv


In [33]:
# Functions to create graphs from JSON files

def json_to_graph(data, graph=None, parent=None):
    """
    Recursively traverse a JSON structure and create a graph with nodes and edges.

    Parameters:
    - data: dict or list, the JSON data to be processed
    - graph: nx.Graph or nx.DiGraph, the graph being built
    - parent: str, the parent node (if any) for creating edges

    Returns:
    - graph: a NetworkX graph representing the JSON structure.
    """
    if graph is None:
        graph = nx.DiGraph()

    if isinstance(data, dict):
        for key, value in data.items():
            if value is None or value == "":
                continue

            node_id = f"{parent}.{key}" if parent else key
            # Include the key as the label (preserve index notation if present)
            label = key.split('[')[0] + key[key.find('['):] if '[' in key else key
            graph.add_node(node_id, label=label, value=str(value))
            
            if parent:
                graph.add_edge(parent, node_id, relation="contains")
            
            json_to_graph(value, graph, node_id)
    
    elif isinstance(data, list):
        for index, item in enumerate(data):
            if item is None or item == "":
                continue

            node_id = f"{parent}[{index}]" if parent else f"[{index}]"
            label = parent.split('.')[-1] + f"[{index}]"
            graph.add_node(node_id, label=label, value=str(item))
            
            if parent:
                graph.add_edge(parent, node_id, relation="contains")
            
            json_to_graph(item, graph, node_id)
    
    else:
        if data is None or data == "":
            return graph

        node_id = str(parent)
        label = parent.split('.')[-1] if parent else "root"
        graph.add_node(node_id, label=label, value=str(data))
        if parent:
            graph.add_edge(parent.rsplit('.', 1)[0], node_id, relation="contains")
    
    return graph

def json_to_graph_keep_deepest(data, graph=None, parent=None):
    """
    Recursively traverse a JSON structure and create a graph with only the deepest literal nodes.

    Parameters:
    - data: dict or list, the JSON data to be processed
    - graph: nx.Graph or nx.DiGraph, the graph being built
    - parent: str, the parent node (if any) for creating edges

    Returns:
    - graph: a NetworkX graph with only the deepest literal values.
    """
    if graph is None:
        graph = nx.DiGraph()

    if isinstance(data, dict):
        is_leaf = True
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                is_leaf = False
                json_to_graph_keep_deepest(value, graph, f"{parent}.{key}" if parent else key)
            elif value is not None and value != "":
                is_leaf = False
                node_id = f"{parent}.{key}" if parent else key
                # Include the key as the label (preserve index notation if present)
                label = key.split('[')[0] + key[key.find('['):] if '[' in key else key
                graph.add_node(node_id, label=label, value=str(value))
                if parent:
                    graph.add_edge(parent, node_id, relation="contains")

        if is_leaf and parent and parent in graph:
            graph.remove_node(parent)

    elif isinstance(data, list):
        is_leaf = True
        for index, item in enumerate(data):
            if isinstance(item, (dict, list)):
                is_leaf = False
                json_to_graph_keep_deepest(item, graph, f"{parent}[{index}]" if parent else f"[{index}]")
            elif item is not None and item != "":
                is_leaf = False
                node_id = f"{parent}[{index}]" if parent else f"[{index}]"
                label = parent.split('.')[-1] + f"[{index}]"
                graph.add_node(node_id, label=label, value=str(item))
                if parent:
                    graph.add_edge(parent, node_id, relation="contains")

        if is_leaf and parent and parent in graph:
            graph.remove_node(parent)

    else:
        if data is not None and data != "":
            node_id = parent
            label = parent.split('.')[-1] if parent else "root"
            graph.add_node(node_id, label=label, value=str(data))
            if parent:
                graph.add_edge(parent.rsplit('.', 1)[0], node_id, relation="contains")

    return graph

In [34]:
# Function to open a CSV file
def read_csv_file(filename):
    """Reads a CSV file separated by ';' and returns a Pandas DataFrame."""
    try:
        df = pd.read_csv(filename, sep=';', engine='python')  # Explicitly set separator
        return df
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return None
    
# Function to write graph nodes to a CSV file
def save_graph_nodes_to_csv(G, csv_file_path):
    """
    Saves the nodes and their attributes from a NetworkX graph to a CSV file.
    
    Parameters:
        G (networkx.Graph): The input graph.
        csv_file_path (str): The file path where the CSV will be saved.
    
    Returns:
        None
    """
    # Open the file and write the nodes
    with open(csv_file_path, mode="w", newline="") as file:
        writer = csv.writer(file)
        
        # Write the header
        writer.writerow(["Node", "Attribute", "Value"])
        
        # Write each node with its attributes
        for node, attributes in G.nodes(data=True):
            for attr, value in attributes.items():
                writer.writerow([node, attr, value])

    print(f"Graph nodes saved to {csv_file_path}")


In [35]:
# Function to call the API of ChatGPT
# Load environment variables from .env file
load_dotenv()

def call_chatgpt_api(outputNodes, normalized_labels_list, prompt):
    """
    Calls the OpenAI ChatGPT API with a prompt, outputNodes DataFrame, and normalized labels list.
    
    Parameters:
        outputNodes (pd.DataFrame): The DataFrame containing nodes.
        normalized_labels_list (list): The set of unique normalized labels.
        prompt (str): The text prompt for ChatGPT.
        
    Returns:
        str: The ChatGPT response.
    """
    api_key = os.getenv("OPENAI_API_KEY")  # Load API key securely from environment
    if not api_key:
        raise ValueError("Error: API key not found! Ensure it's set in the .env file.")

    client = openai.OpenAI(api_key=api_key)

    # Convert DataFrame and label list to strings for API context
    output_nodes_str = outputNodes.to_string(index=False)  # Convert DataFrame to string
    labels_str = ", ".join(normalized_labels_list)  # Convert list to comma-separated string

    # Combine input into the final prompt
    full_prompt = f"""
    Given the following dataset:
    {output_nodes_str}

    And the following normalized labels:
    {labels_str}

    {prompt}
    """

    # Call OpenAI API (New Syntax)
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": full_prompt}
        ],
        max_tokens=1000
    )

    return response.choices[0].message.content  # Extract and return the response

In [4]:
flowertrucks_json_data = {
    "messageType": "Transport Opdracht",
    "message": {
        "shipment": {
            "shipper": "Asian Export Food",
            "loadingLocation": {
                "plaats": "Kampen",
                "land": "Nederland"
            },
            "loadingDate": "17-01-2025"
        },
        "goods": [
            {
                "kenmerk": "00803055558",
                "actie": "Lossen",
                "euroDV": 1,
                "blokDV": 1,
                "noOfColli": 23,
                "unloadLocation": {
                    "straat": "Masvagen 2",
                    "naam": "AX Utveckling AB",
                    "postcode": "22100",
                    "plaats": "MARIEHAMN",
                    "land": "FIN"
                }
            },
            {
                "kenmerk": "00803055544",
                "actie": "Lossen",
                "euroDV": 1,
                "blokDV": 1,
                "noOfColli": 16,
                "unloadLocation": {
                    "straat": "Hämeenkatu 7",
                    "naam": "Nurmi Dao Oy",
                    "postcode": "20500",
                    "plaats": "TURKU",
                    "land": "FIN"
                }
            },
            {
                "kenmerk": "0080305561",
                "actie": "Lossen",
                "euroDV": 1,
                "blokDV": 1,
                "noOfColli": 1,
                "unloadLocation": {
                    "straat": "Kuunsade 10b 43",
                    "naam": "Golden Food Oy",
                    "postcode": "02210",
                    "plaats": "HELSINKI",
                    "land": "FIN"
                }
            },
            {
                "kenmerk": "00803055599",
                "actie": "Lossen",
                "euroDV": 2,
                "blokDV": 3,
                "noOfColli": 50,
                "unloadLocation": {
                    "straat": "Gatan 5",
                    "naam": "ABC Logistics",
                    "postcode": "11450",
                    "plaats": "STOCKHOLM",
                    "land": "SWE"
                }
            },
            {
                "kenmerk": "0080305560",
                "actie": "Lossen",
                "euroDV": 1,
                "blokDV": 2,
                "noOfColli": 12,
                "unloadLocation": {
                    "straat": "Storgata 15",
                    "naam": "Nordic Foods",
                    "postcode": "0161",
                    "plaats": "OSLO",
                    "land": "NOR"
                }
            },
            {
                "kenmerk": "0080305562",
                "actie": "Lossen",
                "euroDV": 2,
                "blokDV": 3,
                "noOfColli": 25,
                "unloadLocation": {
                    "straat": "Skolgatan 12",
                    "naam": "Food Service AB",
                    "postcode": "11850",
                    "plaats": "GOTHENBURG",
                    "land": "SWE"
                }
            },
            {
                "kenmerk": "0080305563",
                "actie": "Lossen",
                "euroDV": 1,
                "blokDV": 2,
                "noOfColli": 10,
                "unloadLocation": {
                    "straat": "Bryggen 3",
                    "naam": "North Sea Logistics",
                    "postcode": "5003",
                    "plaats": "BERGEN",
                    "land": "NOR"
                }
            },
            {
                "kenmerk": "0080305564",
                "actie": "Lossen",
                "euroDV": 2,
                "blokDV": 1,
                "noOfColli": 8,
                "unloadLocation": {
                    "straat": "Tallinn Road 8",
                    "naam": "Baltic Foods",
                    "postcode": "10111",
                    "plaats": "TALLINN",
                    "land": "EST"
                }
            },
            {
                "kenmerk": "0080305565",
                "actie": "Lossen",
                "euroDV": 2,
                "blokDV": 3,
                "noOfColli": 18,
                "unloadLocation": {
                    "straat": "Main Street 22",
                    "naam": "Global Goods Inc",
                    "postcode": "8000",
                    "plaats": "AARHUS",
                    "land": "DNK"
                }
            },
            {
                "kenmerk": "0080305566",
                "actie": "Lossen",
                "euroDV": 1,
                "blokDV": 2,
                "noOfColli": 15,
                "unloadLocation": {
                    "straat": "Food Street 45",
                    "naam": "ScandiFoods AS",
                    "postcode": "3010",
                    "plaats": "DRAMMEN",
                    "land": "NOR"
                }
            }
        ]
    }
}
# Generate the graph
Flowertrucks_G = json_to_graph(flowertrucks_json_data)
Flowertrucks_G2 = json_to_graph_keep_deepest(flowertrucks_json_data)

In [5]:
# List all nodes with their attributes
print("Nodes:")
for node, attributes in Flowertrucks_G.nodes(data=True):
    print(f"  {node}: {attributes}")

# List all edges with their relationships
print("\nEdges:")
for source, target, attributes in Flowertrucks_G.edges(data=True):
    print(f"  {source} -> {target} [relation: {attributes['relation']}]")

Nodes:
  messageType: {'label': 'messageType', 'value': 'Transport Opdracht'}
  message: {'label': 'message', 'value': "{'shipment': {'shipper': 'Asian Export Food', 'loadingLocation': {'plaats': 'Kampen', 'land': 'Nederland'}, 'loadingDate': '17-01-2025'}, 'goods': [{'kenmerk': '00803055558', 'actie': 'Lossen', 'euroDV': 1, 'blokDV': 1, 'noOfColli': 23, 'unloadLocation': {'straat': 'Masvagen 2', 'naam': 'AX Utveckling AB', 'postcode': '22100', 'plaats': 'MARIEHAMN', 'land': 'FIN'}}, {'kenmerk': '00803055544', 'actie': 'Lossen', 'euroDV': 1, 'blokDV': 1, 'noOfColli': 16, 'unloadLocation': {'straat': 'Hämeenkatu 7', 'naam': 'Nurmi Dao Oy', 'postcode': '20500', 'plaats': 'TURKU', 'land': 'FIN'}}, {'kenmerk': '0080305561', 'actie': 'Lossen', 'euroDV': 1, 'blokDV': 1, 'noOfColli': 1, 'unloadLocation': {'straat': 'Kuunsade 10b 43', 'naam': 'Golden Food Oy', 'postcode': '02210', 'plaats': 'HELSINKI', 'land': 'FIN'}}, {'kenmerk': '00803055599', 'actie': 'Lossen', 'euroDV': 2, 'blokDV': 3, 'no

In [ ]:
save_graph_nodes_to_csv(G=Flowertrucks_G, csv_file_path = "graph_nodes.csv")


In [ ]:
save_graph_nodes_to_csv(G=Flowertrucks_G2, csv_file_path = "deep_graph_nodes.csv")


In [ ]:
unique_labels = set()

for node, attributes in Flowertrucks_G.nodes(data=True):
    if "label" in attributes and "value" in attributes:
        if not isinstance(attributes["value"], (dict, list)):
            unique_labels.add(attributes["label"])  # Add to set to ensure uniqueness

# Convert to a sorted list (optional)
unique_labels_list = sorted(unique_labels)

normalized_labels = set()
for label in unique_labels:
    # Use regex to remove the `[n]` part dynamically
    normalized_label = re.sub(r"\[\d+\]$", "", label)
    normalized_labels.add(normalized_label)  # Ensure uniqueness

# Convert to sorted list (optional, for readability)
normalized_labels_list = sorted(normalized_labels)

# Print the cleaned-up labels
print(normalized_labels_list)

['actie', 'blokDV', 'euroDV', 'goods', 'kenmerk', 'land', 'loadingDate', 'loadingLocation', 'message', 'messageType', 'naam', 'noOfColli', 'plaats', 'postcode', 'shipment', 'shipper', 'straat', 'unloadLocation']


In [ ]:
outputNodes = read_csv_file('outputNodes.csv')
outputNodes.head(10)


,Concept,Unnamed: 1,Unnamed: 2,Property,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,Container,UUID,equipmentTypeCode,containerNumber,containerSize,containerType,sealIndicator,isEmpty,isFull,equipmentProperties,equipmentCategoryCode,equipmentContainerITUCode,hasNumberOfCollies,NaN,NaN,NaN,NaN
1,Goods,UUID,goodsTypeCode,typeOfCargo,natureOfCargo,PackageTypeNumericCode,packageCode,packageTypeName,shippingMarks,numberOfTEU,numberofPackages,goodsDescription,goodsNumbers,NaN,NaN,NaN,NaN
2,BusinessActivity,UUID,hasDocumentType,documentID,documentVersion,hasMRN_Number,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Vessel,UUID,vesselName,transportMeansMode,vesselType,voyageNumber,vesselId,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Truck,UUID,hasVIN,hasTransportmeansNationality,transportMeansMode,truckLicensePlate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Wagon,UUID,wagonBrakeType,wagonBrakeWeight,wagonMaximumSpeed,wagonNrAxel,wagonId,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,DangerousGoods,UUID,hazardouseMaterialDetail,UNDGCode,dangerousGoodsRegulationCode,hazmatUNCode,packageLevelType,packageProperties,packageTypeNumericCode,packagingDangerLevelCode,hasFlashpointTemperature,productName,productProperties,properShippingName,hasDangerousGoodsRegulationCode,NaN,NaN
7,Locomotive,UUID,locomotiveBrakeType,locomotiveBrakeWeight,locomotiveDriverIndication,locomotiveTractionType,locomotiveTypeNumber,transportMeansMode,locomotiveId,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Trailer,UUID,trailerLicensePlate,trailerId,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Seal,UUID,sealProperties,sealQuantity,hasSealConditionCode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
prompt = "You are a data engineer with special knowledge on knowledge representation, data transformation and linked data. Take the outputNodes, normalized labels list, and try to replace the values in the normalized labels list with the term that is most suited from the outputNodes file. Give me as output the replaced terms in the normalized labels list. Also tell me your reasoning."

In [30]:
response = call_chatgpt_api(outputNodes, normalized_labels_list, prompt)

In [31]:
print(response)

By analyzing the data, we can see that the normalized labels represent abstract terms, which can be more specifically detailed using the appropriate terms from your provided dataset. Here are my matches based on the information given:

- actie - (No matching term available from the given dataset)
- blokDV - (No matching term available from the given dataset)
- euroDV - (No matching term available from the given dataset)
- goods - Goods
- kenmerk - UUID
- land - legalPersonCountry or country (in Location)
- loadingDate - (No matching term available from the given dataset)
- loadingLocation - PLD:place of delivery or POL:port of loading (in LocationRoles)
- message - (No matching term available from the given dataset)
- messageType - hasDocumentType (in BusinessActivity)
- naam - legalPersonName or vesselName or locomotiveTractionType (depends on context)
- noOfColli - hasNumberOfCollies (in Container)
- plaats - latitude or longitude or name (in Location) or place of acceptance (in Loca